In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sqlalchemy
import numpy as np
from pathlib import Path
import os
import sys
import subprocess
import time
import requests

In [3]:
!pip install ollama --quiet

In [4]:
# Fix paths for src files
project_root = Path(os.getcwd()).parent
script_dir = project_root / "src"
if str(script_dir) not in sys.path:
    sys.path.append(str(script_dir))

# Import optimized processor
from transform.structure_data import process_cbk_files

In [5]:
scratch_env = os.environ.get("SCRATCH")
if scratch_env:
    SCRATCH_DIR = Path(scratch_env)
else:
    raise ValueError("SCRATCH environment variable is not set!")

# Join the paths using the '/' operator
MARKER_OUTPUT_DIR = SCRATCH_DIR / "mshauri-fedha/data/knbs/marker-output"
CBK_CSV_DIR = SCRATCH_DIR / "mshauri-fedha/data/cbk/structured"

# Check if it exists
print(f"Checking path: {CBK_CSV_DIR}")
print(f"Exists: {CBK_CSV_DIR.exists()}")

Checking path: /capstor/scratch/cscs/tligawa/mshauri-fedha/data/cbk/structured
Exists: True


In [6]:
example_file = os.listdir(CBK_CSV_DIR)[5]
example_df = pd.read_csv(os.path.join(CBK_CSV_DIR, example_file))
example_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Process files
# process_cbk_files("mshauri-fedha/data/cbk/structured")

2025-12-22 15:22:29,608 - 🚀 Starting Ollama (qwen2.5:14b)...
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 2049f5674b1e: 100% ▕██████████████████▏ 9.0 GB                         
pulling 66b9ea09bd5b: 100% ▕██████████████████▏   68 B                         
pulling eb4402837c78: 100% ▕██████████████████▏ 1.5 KB                         
pulling 832dd9e00a68: 100% ▕██████████████████▏  11 KB                         
pulling db59b814cab7: 100% ▕██████████████████▏  488 B                         
verifying sha256 digest 
writing manifest 
success 
2025-12-22 15:22:47,378 - ⏩ Skipping cbk_batch_102_191151_december__2019_tap_fxd3_2019_5__dated_23_12_2019_sheet_results.csv (Blacklisted)
2025-12-22 15:22:47,379 - Processing cbk_batch_103_191322_annual_gdp.csv...


🚀 Processing 112 files...


2025-12-22 15:22:55,859 - HTTP Request: POST http://127.0.0.1:25000/api/chat "HTTP/1.1 200 OK"
2025-12-22 15:22:56,168 -    ✅ Saved 25 rows to 'annual_gdp'
2025-12-22 15:22:56,169 - Processing cbk_batch_103_191322_bop_annual.csv...
2025-12-22 15:22:56,815 - HTTP Request: POST http://127.0.0.1:25000/api/chat "HTTP/1.1 200 OK"
2025-12-22 15:22:56,979 -    ✅ Saved 59 rows to 'bop_annual'
2025-12-22 15:22:56,979 - Processing cbk_batch_103_191322_central_bank_rates.csv...
2025-12-22 15:22:57,606 - HTTP Request: POST http://127.0.0.1:25000/api/chat "HTTP/1.1 200 OK"
/users/tligawa/mshauri-fedha/src/transform/structure_data.py:280: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = df[c].apply(lambda x: pd.to_numeric(str(x).replace(',', '').replace('(', '-').replace(')', ''), errors='ignore'))
2025-12-22 15:22:57,957 -    ✅ Saved 412 rows to 'central_bank_rates'
2025-12-22 1


✅ Done.
📊 Created 65 tables.


In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from sqlalchemy import create_engine

In [7]:
# Setup Connection
engine = create_engine("sqlite:///mshauri_fedha_v6.db")

# Get Table List
try:
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", engine)
    table_list = tables['name'].tolist()
except:
    table_list = []
    print("❌ Could not read database. Check DB_NAME path.")

In [8]:
# Create Widgets
dropdown = widgets.Dropdown(
    options=table_list,
    description='Select Table:',
    disabled=False,
)

out = widgets.Output()

In [9]:
# Define Logic to observe tables
def on_table_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        table_name = change['new']
        with out:
            clear_output() # Clear previous result
            print(f"Loading '{table_name}'...")
            try:
                # Get schema info
                df_head = pd.read_sql(f'SELECT * FROM "{table_name}" LIMIT 25', engine)
                df_count = pd.read_sql(f'SELECT COUNT(*) as count FROM "{table_name}"', engine).iloc[0]['count']
                
                print(f"✅ Total Rows: {df_count}")
                display(df_head)
            except Exception as e:
                print(f"Error loading table: {e}")

# Display UI
dropdown.observe(on_table_change)
display(dropdown, out)

# Trigger first load if tables exist
if table_list:
    dropdown.value = table_list[0]

Dropdown(description='Select Table:', options=('annual_gdp', 'cbk_indicative_exchange_rates_2003', 'cbk_indica…

Output()

In [5]:
import logging
from load.clean_db import clean_database_pipeline

In [13]:
# Configure Logging to see output in the notebook
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(message)s',
    stream=sys.stdout,
    force=True # Ensures notebook captures the logs
)

# Run the pipeline
DB_NAME = "mshauri_fedha_v6.db" 

clean_database_pipeline(DB_NAME)

2025-12-22 15:58:29,433 - 🧹 Starting cleanup on mshauri_fedha_v6.db...
2025-12-22 15:58:29,497 - 🗑️  Dropping 17 tables...
2025-12-22 15:58:29,575 -    - Dropped: bop_annual
2025-12-22 15:58:30,071 -    - Dropped: depository_corporation_survey
2025-12-22 15:58:30,272 -    - Dropped: exchange_rates_end_period
2025-12-22 15:58:30,397 -    - Dropped: exchange_rates_period_average
2025-12-22 15:58:30,510 -    - Dropped: forex_bureau_rates_sheet_fbx2
2025-12-22 15:58:30,682 -    - Dropped: forex_bureau_rates_sheet_fbx1
2025-12-22 15:58:30,804 -    - Dropped: forex_bureau_rates_sheet_market_intelligence
2025-12-22 15:58:30,920 -    - Dropped: forex_bureau_rates_sheet_chief_dealer
2025-12-22 15:58:31,089 -    - Dropped: forex_bureau_rates_sheet_director
2025-12-22 15:58:31,196 -    - Dropped: forex_bureau_rates_sheet_fxb1
2025-12-22 15:58:31,325 -    - Dropped: forex_bureau_rates_sheet_fxb2
2025-12-22 15:58:31,437 -    - Dropped: forex_bureau_rates_sheet_sheet1
2025-12-22 15:58:31,604 -    - 

In [14]:
from sqlalchemy import text

engine = create_engine(f"sqlite:///{DB_NAME}")

print("📊 Verifying 'foreign_trade_summary' (Should have dropped 1 row):")
df = pd.read_sql('SELECT * FROM "foreign_trade_summary" LIMIT 5', engine)
display(df)

print("\n📊 Verifying 'indicative_rates_sheet_press' (Should have 'bank_name', 'usd_buy'...):")
df = pd.read_sql('SELECT * FROM "indicative_rates_sheet_press" LIMIT 5', engine)
display(df)

📊 Verifying 'foreign_trade_summary' (Should have dropped 1 row):


,kenya_shillings_million_year,month,commercial_imports_cif,government_imports_cif,total_direct_imports_cif,domestic_fob,re_exports_fob,total_fob,trade_balance,source_file
0,1998,8,"14,786.66",736.34,"15,522.99","8,642.42",506.74,"9,149.16","-6,373.83",cbk_batch_103_191322_foreign_trade_summary.csv
1,1998,9,"16,438.24",511.11,"16,949.35","9,222.37",639.03,"9,861.40","-7,087.95",cbk_batch_103_191322_foreign_trade_summary.csv
2,1998,10,"16,189.14",980.93,"17,170.07","9,310.95",572.66,"9,883.61","-7,286.46",cbk_batch_103_191322_foreign_trade_summary.csv
3,1998,11,"13,424.42",395.24,"13,819.65","8,633.71",455.17,"9,088.89","-4,730.77",cbk_batch_103_191322_foreign_trade_summary.csv
4,1998,12,"17,529.81",516.13,"18,045.94","9,079.32",517.33,"9,596.66","-8,449.28",cbk_batch_103_191322_foreign_trade_summary.csv



📊 Verifying 'indicative_rates_sheet_press' (Should have 'bank_name', 'usd_buy'...):


,date,currency,buy_rate_usd,sell_rate_usd,margin_usd,buy_rate_gbp,sell_rate_gbp,margin_gbp,buy_rate_eur,sell_rate_eur,margin_eur,bank_name,total_margin,notes,extra_data,source_file
0,A B C,103.60000000000001,103.80,0.2,136.74,137.02,0.28,121.91,122.15,0.24,,,,,,cbk_batch_32_172348_indicative_rates_sheet_PRE...
1,EQUITY,103.5,103.70,0.2,136.65,136.95,0.30,121.92,122.17,0.25,,,,,,cbk_batch_32_172348_indicative_rates_sheet_PRE...
2,I & M,103.60000000000001,103.80,0.2,136.74,137.02,0.28,121.91,122.16,0.25,,,,,,cbk_batch_32_172348_indicative_rates_sheet_PRE...
3,DIAMOND TRUST,103.56,103.76,0.2,136.68,136.97,0.29,121.85000000000001,122.13,0.28,,,,,,cbk_batch_32_172348_indicative_rates_sheet_PRE...
4,N I C,103.65,103.85,0.2,136.8,137.07,0.27,121.85000000000001,122.09,0.24,,,,,,cbk_batch_32_172348_indicative_rates_sheet_PRE...


In [6]:
from load.clean_db import drop_tables, fix_foreign_trade, fix_indicative_rates_shift, fix_cbk_indicative_swap

In [20]:
# --- CONFIGURATION ---
DB_NAME = "mshauri_fedha_v6.db"
DB_CONNECTION = f"sqlite:///{DB_NAME}"
engine = create_engine(DB_CONNECTION)

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger()


# --- EXECUTION ---
print(f"🔧 Starting Clean-Up on {DB_NAME}...\n")

drop_tables(engine)
fix_foreign_trade(engine)
fix_indicative_rates_shift(engine)
fix_cbk_indicative_swap(engine)

print("\n✨ Operations Complete.")

🔧 Starting Clean-Up on mshauri_fedha_v6.db...

🗑️  Dropping Tables...
   - Dropped: forex_bureau_rates
   - Dropped: forex_bureaus_rates_sheet_chief_dealers
   - Dropped: forex_bureaus_rates_sheet_director
   - Dropped: forex_bureaus_rates_sheet_directors
   - Dropped: forex_bureaus_rates_sheet_fbx
   - Dropped: forex_bureaus_rates_sheet_fbx1
   - Dropped: forex_bureaus_rates_sheet_fbx2
   - Dropped: forex_bureaus_rates_sheet_fxb1
   - Dropped: forex_bureaus_rates_sheet_fxb2
   - Dropped: forex_bureaus_rates_sheet_fxb22
   - Dropped: forex_bureaus_rates_sheet_market_intelligence
   - Dropped: forex_bureaus_rates_sheet_sheet1
   - Dropped: forex_bureaus_rates_sheet_sheet2
   - Dropped: forex_bureaus_rates_sheet_sheet3
   - Dropped: forex_bureaus_rates_sheet_sheet4
   - Dropped: issues_of_treasury_bills
   - Dropped: issues_of_treasury_bonds
ℹ️  'foreign_trade_summary': Target column not found.
✅ Fixed 'indicative_rates_sheet_indicative': Applied Date Shift & Header Rename.
✅ Fixed 'indi

In [25]:
# Connect to Database
db_name = "mshauri_fedha_v6.db"
engine = create_engine(f"sqlite:///{db_name}")

print(f"🔧 Connecting to {db_name}...")

# Drop the specific tables
with engine.connect() as conn:
    tables_to_drop = [
        "indicative_rates_sheet_indicative",
        "commercial_banksaverage_lending_rates_2015"
    ]
    
    for t in tables_to_drop:
        conn.execute(text(f'DROP TABLE IF EXISTS "{t}"'))
        print(f"   🗑️  Dropped table: {t}")
    
    conn.commit()

# Rename column in 'foreign_trade_summary'
try:
    # Read the table
    df = pd.read_sql('SELECT * FROM "foreign_trade_summary"', engine)
    
    # Check if the target column exists
    target_col = "kenya_shillings_million_year"
    if target_col in df.columns:
        # Rename
        df.rename(columns={target_col: "year"}, inplace=True)
        
        # Save back (Overwriting the old table)
        df.to_sql("foreign_trade_summary", engine, if_exists="replace", index=False)
        print(f"   ✅ Renamed '{target_col}' to 'year' in 'foreign_trade_summary'")
    else:
        print(f"   ℹ️  Column '{target_col}' not found in 'foreign_trade_summary'. It might be renamed already.")

except Exception as e:
    print(f"   ❌ Error updating foreign_trade_summary: {e}")

print("\n✨ Done.")

🔧 Connecting to mshauri_fedha_v6.db...
   🗑️  Dropped table: indicative_rates_sheet_indicative
   🗑️  Dropped table: commercial_banksaverage_lending_rates_2015
   ✅ Renamed 'kenya_shillings_million_year' to 'year' in 'foreign_trade_summary'

✨ Done.


In [35]:
from transform.get_csv_from_md import ingest_knbs_data

In [36]:
# Define parameters
KNBS_DIR = os.path.join(SCRATCH_DIR, "mshauri-fedha/data/knbs/marker-output")
DB_NAME = "mshauri_fedha_v6.db"

# Run the Ingestion
ingest_knbs_data(
    input_dir=KNBS_DIR, 
    db_name=DB_NAME, 
    model="qwen2.5:14b"
)

2025-12-22 16:29:06,321 - ✅ Ollama connected.
2025-12-22 16:29:06,879 - 🚀 Found 574 KNBS markdown files (Recursive Scan). Starting ingestion...
2025-12-22 16:29:06,879 - 📄 Processing knbs_batch_100_194427_kenya-quarterly-gross-domestic-product-first-quart.md...
2025-12-22 16:29:06,907 -    found 4 tables.
2025-12-22 16:29:19,463 - HTTP Request: POST http://127.0.0.1:25000/api/chat "HTTP/1.1 200 OK"
2025-12-22 16:29:19,604 -      -> Saved table: knbs_knbs_batch_100_194427_kenya_quarterly_gr_tab0 (19 rows)


<string>:39: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead


2025-12-22 16:29:20,468 - HTTP Request: POST http://127.0.0.1:25000/api/chat "HTTP/1.1 200 OK"
2025-12-22 16:29:20,599 -      -> Saved table: knbs_knbs_batch_100_194427_kenya_quarterly_gr_tab1 (26 rows)


<string>:39: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead


2025-12-22 16:29:21,912 - HTTP Request: POST http://127.0.0.1:25000/api/chat "HTTP/1.1 200 OK"
2025-12-22 16:29:22,064 -      -> Saved table: knbs_knbs_batch_100_194427_kenya_quarterly_gr_tab2 (26 rows)


...

<string>:39: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead


2025-12-22 16:48:36,601 -      -> Saved table: knbs_knbs_batch_1_192735_kenya_leading_econom_tab23 (17 rows)
2025-12-22 16:48:36,603 - LLM Header clean failed (Table 24): Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


<string>:39: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead


2025-12-22 16:48:37,173 -      -> Saved table: knbs_knbs_batch_1_192735_kenya_leading_econom_tab24 (16 rows)
2025-12-22 16:48:37,175 - LLM Header clean failed (Table 25): Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download
2025-12-22 16:48:37,615 -      -> Saved table: knbs_knbs_batch_1_192735_kenya_leading_econom_tab25 (28 rows)
2025-12-22 16:48:37,617 - LLM Header clean failed (Table 26): Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


<string>:39: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead

KeyboardInterrupt



In [40]:
# Connect
engine = create_engine(f"sqlite:///{db_name}")

print(f"🧹 Scanning for KNBS tables in {db_name}...")

with engine.connect() as conn:
    # Find all tables starting with 'knbs_'
    query = text("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE 'knbs_%'")
    knbs_tables = [t[0] for t in conn.execute(query).fetchall()]
    
    if not knbs_tables:
        print("✅ No KNBS tables found.")
    else:
        print(f"found {len(knbs_tables)} tables to remove.")
        for t in knbs_tables:
            conn.execute(text(f'DROP TABLE "{t}"'))
            print(f"   🗑️  Dropped: {t}")
        conn.commit()

print("✨ Cleanup Complete.")

🧹 Scanning for KNBS tables in mshauri_fedha_v6.db...
found 349 tables to remove.
   🗑️  Dropped: knbs_knbs_batch_100_194427_kenya_quarterly_gr_tab4
   🗑️  Dropped: knbs_knbs_batch_100_194427_kenya_quarterly_gr_tab0
   🗑️  Dropped: knbs_knbs_batch_100_194427_kenya_quarterly_gr_tab1
   🗑️  Dropped: knbs_knbs_batch_100_194427_kenya_quarterly_gr_tab2
   🗑️  Dropped: knbs_knbs_batch_100_194427_kenya_quarterly_gr_tab3
   🗑️  Dropped: knbs_knbs_batch_100_194427_producer_price_ind_tab0
   🗑️  Dropped: knbs_knbs_batch_100_194427_producer_price_ind_tab1
   🗑️  Dropped: knbs_knbs_batch_100_194427_producer_price_ind_tab2
   🗑️  Dropped: knbs_knbs_batch_100_194427_producer_price_ind_tab3
   🗑️  Dropped: knbs_knbs_batch_100_194427_producer_price_ind_tab4
   🗑️  Dropped: knbs_knbs_batch_100_194427_producer_price_ind_tab5
   🗑️  Dropped: knbs_knbs_batch_100_194427_producer_price_ind_tab6
   🗑️  Dropped: knbs_knbs_batch_100_194427_producer_price_ind_tab7
   🗑️  Dropped: knbs_knbs_batch_100_194427_produ